<a href="https://colab.research.google.com/github/alfreddey/smart-food-analyzer-/blob/main/training-script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
data_dir = "/content/drive/MyDrive/tfds-food-101"

In [ ]:
(train_ds, val_ds, test_ds), ds_info = tfds.load(
    'food101',
    split=['train', 'validation', "validation[50%:]"],
    as_supervised=True,
    shuffle_files=True,
    data_dir=data_dir,
    with_info=True
)

In [ ]:
print(f"The number of training samples {train_ds.cardinality()}")
print(f"The number of validation samples {val_ds.cardinality()}")
print(f"The number of test samples {test_ds.cardinality()}")

The number of training samples 75750
The number of validation samples 25250
The number of test samples 12625


In [ ]:
img_size = (224, 224, 3)
batch_size = 32
epochs = 20
num_class = 101

In [ ]:
resize = tf.keras.layers.Resizing(224, 224)

In [ ]:
#resize datasets
train_ds = train_ds.map(lambda x, y: (resize(x), y))
val_ds = val_ds.map(lambda x, y: (resize(x), y))
test_ds = test_ds.map(lambda x, y: (resize(x), y))

In [ ]:
train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=img_size,
    include_top=False,
    classifier_activation='softmax'
)

base_model.trainable = False


inputs = tf.keras.layers.Input(shape=img_size)

scale_layer = tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)

x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(num_class, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

model.summary(show_trainable=True)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_1 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ rescaling (Rescaling)       │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ mobilenetv2_1.00_224        │ (None, 7, 7, 1280)    │  2,257,984 │   N   │
│ (Functional)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d    │ (None, 1280)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 1280)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 101)           │    129,381 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 2,387,365 (9.11 MB)

 Trainable params: 129,381 (505.39 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
earlystop_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

print("Fitting the top layer of the model")
model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[earlystop_cb])

Fitting the top layer of the model
Epoch 1/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 391s 105ms/step - accuracy: 0.3838 - loss: 2.5978 - val_accuracy: 0.6018 - val_loss: 1.5168
Epoch 2/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 205s 86ms/step - accuracy: 0.5648 - loss: 1.7034 - val_accuracy: 0.6135 - val_loss: 1.4605
Epoch 3/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 203s 85ms/step - accuracy: 0.5930 - loss: 1.5784 - val_accuracy: 0.6143 - val_loss: 1.4724
Epoch 4/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 240s 101ms/step - accuracy: 0.6014 - loss: 1.5252 - val_accuracy: 0.6218 - val_loss: 1.4463
Epoch 5/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 227s 85ms/step - accuracy: 0.6135 - loss: 1.4791 - val_accuracy: 0.6182 - val_loss: 1.4671
Epoch 6/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 260s 85ms/step - accuracy: 0.6164 - loss: 1.4697 - val_accuracy: 0.6236 - val_loss: 1.4450
Epoch 7/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 240s 100ms/step - accuracy: 0.6208 - loss: 1.4370 - val_accuracy: 0.6190 - val_loss: 1.4647
Epoch 8/20
2368/2368 ━━━━━━━━━━

In [ ]:
model.evaluate(test_ds)

395/395 ━━━━━━━━━━━━━━━━━━━━ 27s 67ms/step - accuracy: 0.6208 - loss: 1.4351


[1.446207880973816, 0.6212277412414551]

In [ ]:
base_model.trainable = True;

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [ ]:
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[earlystop_cb])

Epoch 1/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 379s 139ms/step - accuracy: 0.3973 - loss: 2.7237 - val_accuracy: 0.6164 - val_loss: 1.4493
Epoch 2/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 305s 128ms/step - accuracy: 0.5911 - loss: 1.5613 - val_accuracy: 0.6589 - val_loss: 1.2665
Epoch 3/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 321s 127ms/step - accuracy: 0.6426 - loss: 1.3343 - val_accuracy: 0.6845 - val_loss: 1.1670
Epoch 4/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 307s 128ms/step - accuracy: 0.6771 - loss: 1.1947 - val_accuracy: 0.6996 - val_loss: 1.1008
Epoch 5/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 358s 144ms/step - accuracy: 0.7078 - loss: 1.0737 - val_accuracy: 0.7108 - val_loss: 1.0600
Epoch 6/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 305s 128ms/step - accuracy: 0.7249 - loss: 0.9971 - val_accuracy: 0.7152 - val_loss: 1.0368
Epoch 7/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 322s 128ms/step - accuracy: 0.7494 - loss: 0.9045 - val_accuracy: 0.7212 - val_loss: 1.0124
Epoch 8/20
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 360s 144ms/step - ac

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-17-3553276507.py", line 1, in <cell line: 0>
    model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[earlystop_cb])
  File "/usr/local/lib/python3.11/dist-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py", line 371, in fit
    logs = self.train_function(iterator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py", line 219, in function
    opt_outputs = multi_step_on_iterator(iterator)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/u

TypeError: object of type 'NoneType' has no len()

In [ ]:
print('Evaluation After fine-tuning')
model.evaluate(test_ds)

Evaluation After fine-tuning
395/395 ━━━━━━━━━━━━━━━━━━━━ 24s 60ms/step - accuracy: 0.7239 - loss: 0.9974


[1.0005781650543213, 0.7234059572219849]

In [ ]:
tf.saved_model.save(model, 'model')

In [ ]:
!pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.7.1 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
google-cloud-bigquery 3.35.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    model/ \
    web_model

/bin/bash: line 1: tensorflowjs_converter: command not found
